<a href="https://colab.research.google.com/github/noahruiz416/Health_Data/blob/main/HD_ETL_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Health Data Cleaning + Processing Script

### - This script processes survey data and apple watch health data and combines the two into one pandas dataframe merged by date. 

### - In doing so this script aids my personal health data project by cleaning all collected data which will be used for analysis in the coming months and years.

##### - More info on the project itself and results can be found on the github readme

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Final Test with survey data 
HD = pd.read_csv("gdrive/My Drive/HD_Project/HD.csv")
surveyTest = pd.read_excel("sheetdata")

#cleaning our suvey data
cleanSurvey = surveyTest.drop(columns = ['Timestamp'])
cleanSurvey.columns.values[[0, 1 ,2, 3, 4, 5]] = ["homework_stress", "day_rating",
                                               'cups_coffee', 'workout', 
                                                'count_meals', 'hours_sleep']
#cleaning our healthData 
cleanHD = HD.drop(columns = ['Respiratory Rate (count/min)'])
cleanHD = cleanHD.loc[(cleanHD['Date'] >= '2022-02-19 00:00:00')]
cleanHD['Date'] = pd.to_datetime(cleanHD['Date'])

#concatenating our health data into a final dataframe
cleanHD = cleanHD.reset_index().drop(columns='index')
finalDF = pd.concat([cleanHD, cleanSurvey], axis=1)

In [ ]:
#functions that will detect visualize and graph outliers 
def count_outliers (df_column, num_stdev = 3):
    std = np.std(df_column)
    lower_bound = np.mean(df_column) - (std * num_stdev)
    upper_bound = np.mean(df_column) + (std * num_stdev)
    return [1 if value >= upper_bound or value <= lower_bound else 0 for value in df_column]

#detecting outliers with IQR
def IQR(column): 
    median = np.median(column)
    quartile_1 = np.quantile(column, .25)
    quartile_3 = np.quantile(column, .75)
    IQR = quartile_3 - quartile_1
    ue = quartile_3 + (1.5 * IQR)
    le = quartile_1 - (1.5 * IQR)
    return [1 if value >= ue or value <= le else 0 for value in column]
    
#used to detect return a summary of outliers in our dataset and which columns they are located in
#before using must, input only numeric data 
def outliers_summary(df, std_dev = 3):
    for i in df:
        try:
            out = sum(count_outliers(df[i], std_dev))
            print(f"{i.upper()}: {out}")
        except TypeError:
            print(f"{i.upper()} IS A STRING / OBJECT VALUE, FIX OR IGNORE")

def quart_summary(df):
    for i in df:
        try:
            out = sum(IQR(df[i]))
            print(f"{i}: {out}")
        except TypeError:
            print(f"{i} IS A STRING / OBJECT VALUE, FIX OR IGNORE")

In [ ]:
finalDF = finalDF.dropna(axis = 1, how='any')